# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and Pandas skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and save the information in a csv
* Perform pagination to retrieve troves of data!
* Write Pandas code to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the data questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Plan out what . One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Save the data you pull as a csv

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Save the data you pull as a csv

6. Take all of the business IDs from your business search, and  using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and save them in a csv.

7. Write Pandas code to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your files.


 
## Data Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to think about what tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your Pandas Dataframes. 


https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [1]:
import numpy as np
import pandas as pd

import requests
import json
import os.path

from  keys  import  client_id, api_key

In [2]:
# Write a function to make a call to the yelp API

In [3]:
def biz_call(term, location, url_params):
    url =  'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': f'Bearer {api_key}'}
    response = requests.get(url, headers=headers, params=url_params)
    if response.status_code == 200:
        response = json.loads(response.text)
        return response
    else:
        print('Status Code: ' + str(response.status_code))
        return None

In [4]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

In [5]:
def biz_parse(results):
    parsed_data = []
    for result in results['businesses']:
        data = {
            'Business Name': result['name'],
            'Business ID': result['id'],
            'Review Count': result['review_count'],
            'Business Rating': result['rating'],
        }
        if 'price' in list(result.keys()):
            data['Price Range'] = result['price']
        else:
            data['Price Range'] = np.nan
        parsed_data.append(data)
    return parsed_data

In [6]:
# Write a function to take your parsed data and insert it into CSV

In [7]:
    # your code to save the current results with all of the other results. 
    # I would save the data every time you pull 50 results
    # in case something breaks in the process.

In [8]:
def data_save(parsed_results, columns, filename):
    df = pd.read_csv(filename)
    new_df = pd.DataFrame(parsed_results, columns=columns)
    combined_df = pd.concat([df, new_df], join='outer')
    combined_df.to_csv(filename, index=False)
    return None

In [9]:
# Write a script that combines the three functions above into a single process.
# create a variable  to keep track of which result you are in. 

#set up a while loop to go through and grab the result 
    #set the offset parameter to be where you currently are in the results 
    #make your API call with the new offset number
    #after you get your results you can now use your function to parse those results
    # use your function to insert your parsed results into the db
    #increment the counter by 50 to move on to the next results

In [40]:
def yelp_biz_search(term, location, biz_file):
    filename = biz_file
    columns = ['Business Name', 'Business ID', 'Review Count',
               'Business Rating', 'Price Range']
    df = pd.DataFrame(columns=columns)
    df.to_csv(filename, index=False)
    total_results = 1
    offset = 0
    while offset < total_results and offset < 1000:
        url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': 50,
        'offset': offset
        }
        results = biz_call(term, location, url_params)
        parsed_results = biz_parse(results)
        data_save(parsed_results, columns, filename)
        total_results = results['total']
        offset += 50
    total_saved = len(pd.read_csv(filename))
    print(f'Total Results: {total_results}\nTotal Saved: {total_saved}')
    return None

##### Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [11]:
# write Pandas code to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [12]:
# write a function that takes a business id 
# and makes a call to the API for reivews

In [13]:
def reviews_call(biz_id):
    url = f'https://api.yelp.com/v3/businesses/{biz_id}/reviews'
    headers = {'Authorization': f'Bearer {api_key}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        response = json.loads(response.text)
        return response
    else:
        print('Status Code: ' + str(response.status_code))
        return None

In [14]:
# Write a function to parse out the relevant information from the reviews

In [27]:
def reviews_parse(results, biz_name, biz_id):
    parsed_data = []
    if 'reviews' in list(results.keys()):
        for result in results['reviews']:
            data = {
                'Business Name': biz_name,
                'Business ID': biz_id,
                'Review ID': result['id'],
                'Review Text': result['text'],
                'Review Rating': result['rating'],
                'Review Time': result['time_created']
            }
            parsed_data.append(data)
        return parsed_data
    else:
        return None

In [28]:
# combine the functions above into a single script  

In [30]:
def yelp_reviews_search(biz_file, reviews_file):
    filename = reviews_file
    columns = ['Business Name', 'Business ID', 'Review ID',
               'Review Text', 'Review Rating', 'Review Time']
    df = pd.DataFrame(columns=columns)
    df.to_csv(filename, index=False)
    biz_df = pd.read_csv(biz_file)
    for biz_name, biz_id in zip(biz_df['Business Name'], biz_df['Business ID']):
        results = reviews_call(biz_id)
        if results is not None:
            parsed_results = reviews_parse(results, biz_name, biz_id)
            data_save(parsed_results, columns, filename)
        else:
            pass
    total_saved = len(pd.read_csv(filename))
    print(f'Total Saved: {total_saved}')
    return None

## Part 5 -  Write Pandas code that will answer the questions posed. 

Now that your data is saved in CSVs, you can answer the questions. 

In [41]:
# Yelp Business Search

term = 'Bar'
location = 'Albany NY'
biz_file = 'Bar_Albany_Businesses.csv'

yelp_biz_search(term, location, biz_file)

Total Results: 221
Total Saved: 221


In [32]:
# Yelp Reviews Search

biz_file = 'Bar_Albany_Businesses.csv'
reviews_file = 'Bar_Albany_Reviews.csv'

yelp_reviews_search(biz_file, reviews_file)

Total Saved: 294


# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [39]:
pd.read_csv('Bar_Albany_Businesses.csv', index_col='Business Name').sort_index()

,Business ID,Review Count,Business Rating,Price Range
Business Name,,,,
20 North Broadway Tavern,kqYCzp68RAonIITm32q8Fg,121,4.5,$$
288 Lark Wine & Tap,EK1IzMI79YpDy6h7zxsNVw,27,4.5,NaN
Across The Street Pub,2YVy8k9NAZ0vMlrC8xyq8A,61,3.5,$$
Ale House,Qh-hU4JKEjF4kkSe0IGdSA,62,4.5,$
Ama Cocina,A_CVNy2gjuQcSmzxjWwx-w,366,3.5,$$
...,...,...,...,...
Waterworks Pub,CzfqowTGOa55GM41RWo6bQ,24,3.5,$$
Wellington's,UddIOPOCt3of2Fsb0_pzcQ,94,3.5,$$
Wolff's Biergarten,l4tYS_3SVYpk9yPOCHR0TA,214,4.0,$$


In [35]:
pd.read_csv('Bar_Albany_Reviews.csv', index_col='Business Name').sort_index().head(50)

,Business ID,Review ID,Review Text,Review Rating,Review Time
Business Name,,,,,
1 Tippling Place,tzVLTQNUtiCAxUZmEuF0vQ,ZUV5KrSmNXPbi_ar16Bw8A,This place was really fun and the bartenders m...,5,2020-01-27 03:48:24
1 Tippling Place,tzVLTQNUtiCAxUZmEuF0vQ,IdJz8bA8Xixpdi2K0Bh7kQ,Strangest thing about Philly - there's no stan...,3,2019-11-19 12:40:44
1 Tippling Place,tzVLTQNUtiCAxUZmEuF0vQ,pBCDhSIF8aQ4n7gCUrHTtA,This place has potential as the atmosphere is ...,2,2020-02-29 19:23:30
2637brew,Gy6tBn6lrf4AN2Gr1I8Oyg,UeGzHX5Y5CMJJp7kM4A6HA,2637 brew has become one of my favorite go to ...,5,2019-10-10 10:54:43
2637brew,Gy6tBn6lrf4AN2Gr1I8Oyg,B0rThBZzKAoQEVNDHVxL2w,This is mainly a review of their catering but ...,5,2019-12-30 08:21:08
2637brew,Gy6tBn6lrf4AN2Gr1I8Oyg,MdMrbMsJ0Ztdg--RINSpMg,Really great brunch spot in Brewerytown area! ...,4,2020-02-02 14:33:24
2nd Story Brewing,jEasa4Sbzy4NdLyzPPgQyg,6TYHwpQW8lTu-idJnBcHyQ,"5 Chicken Quesadilla: 3-cheese blend, flour to...",5,2020-03-17 05:30:05
2nd Story Brewing,jEasa4Sbzy4NdLyzPPgQyg,6VlTWGP5qk9YQPqXzPLSeA,I was really in the mood for a fried chicken s...,3,2020-04-08 09:59:00
2nd Story Brewing,jEasa4Sbzy4NdLyzPPgQyg,HXoYWVBBaIIohIY1iL7yBg,The one Belgian beer on the list was kicked. \...,2,2019-11-02 18:32:47
